# **Build Your First RecSys**

In this notebook, you will build a simple collaborative filtering-based movie recommendation system.

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## Loading the data

We will use the [MovieLens Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) that consists of movies released on or before July 2017. Data points include metadata about the movies, as well as 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.

For the purpose of this exercise, we will use a subset of 100,000 ratings from 700 users on 9,000 movies.


*movies_metadata.csv* contains information about the movies:

In [ ]:
movies = pd.read_csv("https://raw.githubusercontent.com/evgeniyako-edu/build-your-first-recsys-workshop/main/data/movies_metadata.csv",
                     low_memory=False)
movies = movies.drop([19730, 29503, 35587])
movies['id'] = pd.to_numeric(movies['id'])
movies.head()

*ratings_small.csv* contains the ratings. We can drop the $\texttt{timestamp}$ column as we will not use it in this exercise.

In [ ]:
ratings = pd.read_csv("https://raw.githubusercontent.com/evgeniyako-edu/build-your-first-recsys-workshop/main/data/ratings_small.csv",
                     low_memory=False).drop(["timestamp"], axis=1)
ratings.head()

Let's merge the two tables so that the ratings table also contains the name of the movie:

In [ ]:
ratings = (ratings.merge(movies[["id", "title"]],
                        right_on="id", 
                        left_on="movieId")).drop(["id"], axis=1).sort_values("userId")
ratings.head(10)

## Explorative Data Analysis (EDA)

Let's explore the dataset in a bit more detail. 

Most users rate a couple of movies. However, there are some dedicated users who give ratings to 100+ movies: 

In [ ]:
counts_per_user = ratings.groupby("userId")[["movieId"]].count().reset_index()
counts_per_user.columns = ["userId", "num_movies_rated"]
plt.figure(figsize=(12,5))
plt.hist(counts_per_user["num_movies_rated"], bins=range(0,800,50))
plt.xticks(range(0,800,50))
_ = plt.xlabel("number of movies rated")
_ = plt.ylabel("number of users")

What are the most rated movies in the dataset? Display them together with their average rating.

In [ ]:
ratings_per_movie = ratings\
                    .groupby("title")[["userId", "rating"]]\
                    .agg({"userId": len,
                          "rating": np.mean}).reset_index()
ratings_per_movie.columns = ["original_title", "num_users_rated", "avg_rating"]

ratings_per_movie.sort_values("num_users_rated", ascending=False).head(10)

## Collaborative Filtering

In this section, we will implement a simple collaborative filtering-based recommender which would predict the ratings a user would give to unseen movies. Top-rated candidates can then be suggested to the user.

Our system will predict the unknown ranking as a linear combination of the ratings given to the movie by other users who have seen it. We will weigh each of such ratings by how much the user in question is similar to each of these other users. More spicifically, a prediction a user $u$ would give to the unseen movie $i$ is

$$\hat{R_{ui}} = \bar{R}_u + w_{uv}\sum_{v}\left(R_{vi} - \bar{R}_{v} \right)$$

There, the summation goes over all users $v$ who rated movie $i$, and $\bar{R}$ is an average rating given by a user to all the movies they have rated.

Similarity between users $u$ and $v$ can be measured as Pearson correlation between the ratings vectors from these users, considering only the movies both of them rated:

$$w_{uv} = \frac{\sum_j (R_{uj} - \bar{R}_u )(R_{vj} - \bar{R}_v )}{\sqrt{{\sum_j (R_{uj} - \bar{R}_u )^2}{\sum_j (R_{vj} - \bar{R}_v )^2}}}$$

There, the summation goes over all movies $j$ that were rated by both users $u$ and $v$.

Let's first implement some building blocks and then bring them together in a single collaborative filtering system. 

In [ ]:
def get_ratings_movie(movie_id, ratings):
    """ 
    Get all the ratings data for a given movie
    """
    return ratings[ratings["movieId"] == movie_id]

In [ ]:
def get_user_ratings(user_id, ratings):
    """
    Get all the ratings given by a given user
    """
    return ratings[ratings["userId"] == user_id][["movieId", "rating"]]

In [ ]:
def compute_similarity(user_a, user_b, ratings):
    """
    Compute similarity between the two given users.

    Similarity is defined as "correlation coefficient" 
    between the rating vectors from the two users,
    and is computed on a subset of movies rated by both users.
    """
    # Get movies commonly rated by both user_a and user_b
    common_ratings = pd.merge(get_user_ratings(user_a, ratings), 
                              get_user_ratings(user_b, ratings),
                              on="movieId",
                              how="inner",
                              suffixes=('_a', '_b'))
    
    # Let's not compute similarity if two users 
    # have less then 5 movies in common
    if common_ratings.shape[0] <= 5:
        return 0
    # Otherwise, compute the correlation coefficient
    # (we can compute mean rating across all ratings,
    # but the similarity itself based on the commonly rated movies only)
    else:
        mean_rating_a = ratings[ratings["userId"] == user_a]["rating"].mean()
        mean_rating_b = ratings[ratings["userId"] == user_b]["rating"].mean()
        numenator_a = (common_ratings["rating_a"] - mean_rating_a)
        numenator_b = (common_ratings["rating_b"] - mean_rating_b)
        numerator = np.sum(numenator_a*numenator_b)
        denominator_a = np.sum(common_ratings["rating_a"]-mean_rating_a)**2
        denominator_b = np.sum(common_ratings["rating_b"]-mean_rating_b)**2
        denominator = np.sqrt(denominator_a*denominator_b)
        if denominator == 0:
            return 0
        else:
            return numerator/denominator

In [ ]:
def predict_rating(user_id, movie_id, ratings):
    """ 
    Predict rating that a user user_id would give to movie movie_id 
    using collaborative filtering
    """
    
    # Get all the ratings for the movie in question
    ratings_movie = get_ratings_movie(movie_id, ratings)

    # Compute similarity between the user in question, user_id,
    # and everyone who has rated the movie in question, movie_id
    similarities = {}
    for another_user_id in ratings_movie["userId"]:
        similarity = compute_similarity(user_id, another_user_id, ratings)
        if similarity:
            similarities[user_id] = similarity

    # Compute the predicted rating       
            
    prediction = ratings[ratings["userId"] == user_id]["rating"].mean()
    for user_id, similarity in similarities.items():
        prediction += similarity*ratings[ratings["userId"] == user_id]["rating"].mean()
    
    return prediction

Let's take a look at the first user who has rated only 6 movies. Which of the top 50 most-rated movies can we recommend them to watch?

How does it compare to our recommendations to, for example, user 2?

In [ ]:
ratings[ratings["userId"] == 1]

In [ ]:
def get_title(movie_id, movies):
    return movies[movies["id"] == movie_id]["title"].values[0]

def get_movie_id(movie_title, movies):
    return movies[movies["title"] == movie_title]["id"].values[0]

In [ ]:
user_id = 2
predictions = {}
user_ratings = get_user_ratings(user_id, ratings)
for movie_title in ratings_per_movie.sort_values("num_users_rated", ascending=False).head(50)["original_title"]:
    movie_id = get_movie_id(movie_title, movies)
    if movie_id not in user_ratings["movieId"].values:
        predictions[movie_title] = predict_rating(user_id, movie_id, ratings)

In [ ]:
sorted(predictions.items(), key=lambda x: x[1], reverse=True)